In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('clusters').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [7]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [8]:
data.count()

1501

In [9]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [10]:
train,test = data.randomSplit([0.8,0.2],seed=3)

In [13]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [14]:
model =  als.fit(train)

In [16]:
prediction = model.transform(test)

In [17]:
prediction.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    28| -1.9096284|
|      0|   1.0|    26| -1.0224174|
|      0|   1.0|    27| 0.39007226|
|      3|   2.0|    22|-0.10903515|
|      2|   2.0|     1| 0.87932277|
|      0|   1.0|     6|  1.7085646|
|      1|   1.0|     6|   1.484568|
|      2|   1.0|    16|  2.4745553|
|      1|   1.0|    20| 0.86332417|
|      1|   1.0|     5|   1.123127|
|      0|   1.0|    15|  2.3332133|
|      0|   1.0|    23|  1.9300094|
|      2|   1.0|    23| 0.50074553|
|      4|   1.0|    23|  1.0145658|
|      2|   4.0|    10|  1.4590445|
|      4|   1.0|    24|  1.2206041|
|      3|   1.0|    29| -1.1153568|
|      2|   4.0|    21|   2.523672|
|      4|   1.0|    14|  1.0912721|
|      2|   3.0|     0| 0.31037045|
+-------+------+------+-----------+
only showing top 20 rows



In [18]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [19]:
rmse = evaluator.evaluate(prediction)

In [20]:
print('RMSE')
print(rmse)

RMSE
1.7309149478591066


In [21]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     10|    11|
|     19|    11|
|     22|    11|
|     27|    11|
|     32|    11|
|     39|    11|
|     43|    11|
|     59|    11|
|     61|    11|
|     69|    11|
|     75|    11|
|     82|    11|
|     88|    11|
|     90|    11|
+-------+------+



In [22]:
recommendations = model.transform(single_user)

In [23]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     32|    11|  5.9480658|
|     27|    11|   5.448721|
|     75|    11|   4.408212|
|     39|    11|  3.8548386|
|     59|    11|  3.3001447|
|     90|    11|  3.2582552|
|     88|    11|  2.9543836|
|     19|    11|  2.6316943|
|     61|    11|  1.3803993|
|     43|    11|  1.2631811|
|     82|    11|   0.655509|
|     10|    11| 0.56816006|
|     69|    11|  0.2280631|
|     22|    11|-0.12619877|
+-------+------+-----------+

